# **Aprendizaje Supervisado: Random Forest Classifier**

La idea de este notebook es predecir la severidad de accidentes de tránsito en Estados Unidos utilizando un modelo de aprendizaje supervisado

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

drive.mount('/content/drive') # montamos el drive

# cargamos el dataframe de entrenamiento y el dataframe de testeo
# el dataframe de entrenamiento tiene toda la información, incluyendo la columna 'Severity' (severidad)
# esta es la columna que vamos a entrenar el modelo para predecir: 0=severidad baja; 1=severidad alta.
# en cambio, el dataframe de test no tiene esta columna: son los datos que usamos para generar la predicción que se entrega.
filename_train = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/train.csv' 
filename_test = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/test4alumnxs.csv' 

df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

Mounted at /content/drive


In [ ]:
# acá voy a apllicar un par de transformaciones a los datos para poder tener una mejor prediccion:

#df_train.columns

#df_train['Wind_Direction'] = df_train['Wind_Direction'].replace(['North', 'South', 'East', 'West'],['N', 'S', 'E', 'W']) #normaliza la direccion del viento
df_train['Duracion'] = (pd.to_datetime(df_train['End_Time']) - pd.to_datetime(df_train['Start_Time'])).astype('timedelta64[m]') #nos da una idea de la duracion del accidente

df_train.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Duracion'],
      dtype='object')

In [ ]:
y = np.array(df_train['Severity']) # armo el vector de etiquetas

# vamos a entrenar un modelo usando las siguientes variables:
columnas_modelo = ['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Duracion', 'Distance(mi)']

# extraigo la matriz de features X
X = np.array(df_train[columnas_modelo])

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# pruebo con un random forest
clf = RandomForestClassifier()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_test, roc_train


(0.8751340876463825, 0.9999958153243802)

In [ ]:
# aplico las transformaciones al dataframe de testing asi puedo usar el nuevo modelo:
df_test['Duracion'] = (pd.to_datetime(df_test['End_Time']) - pd.to_datetime(df_test['Start_Time'])).astype('timedelta64[m]') #nos da una idea de la duracion del accidente

In [ ]:
X2 = np.array(df_test[columnas_modelo]) # cargo el dataset de testeo

predicciones = clf.predict_proba(X2)[:, 1] # obtengo el vector de probabilidades
# este es el vector que vamos a entregar como resultado del TP

print(predicciones)

# lo guardo en colab
np.savetxt('predicciones.csv', predicciones, delimiter=",")

[0.13       0.44666667 0.2225     ... 0.08       0.75       0.36      ]


In [ ]:
#aca verifico que el len sea correcto :)

print(len(predicciones))
print(X2.shape[0])

#ahora sí.

11372
11372
